In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

import nltk
from nltk.tokenize import word_tokenize

# to get deterministic output
torch.manual_seed(123)

sys.path.append(os.path.abspath(".."))

### Loading the Dataset

In [2]:
document = ""
with open("../datasets/word_prediction_dataset.txt", "r", encoding="utf-8") as file:
    document = file.read()

### Tokenize the dataset

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Nova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
all_tokens = word_tokenize(document.lower())
print("Total Tokens:", len(all_tokens))
all_tokens[:20]

Total Tokens: 1018


['about',
 'the',
 'program',
 'what',
 'is',
 'the',
 'course',
 'fee',
 'for',
 'data',
 'science',
 'mentorship',
 'program',
 '(',
 'dsmp',
 '2023',
 ')',
 'the',
 'course',
 'follows']

### Build Vocabulary

In [5]:
vocab = {'<unk>': 0}

for token in Counter(all_tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

print("Vocab length:", len(vocab))
list(vocab.items())[:20]

Vocab length: 289


[('<unk>', 0),
 ('about', 1),
 ('the', 2),
 ('program', 3),
 ('what', 4),
 ('is', 5),
 ('course', 6),
 ('fee', 7),
 ('for', 8),
 ('data', 9),
 ('science', 10),
 ('mentorship', 11),
 ('(', 12),
 ('dsmp', 13),
 ('2023', 14),
 (')', 15),
 ('follows', 16),
 ('a', 17),
 ('monthly', 18),
 ('subscription', 19)]

### Convert Text to Numerical Sequence

In [6]:
def text_to_indices(sentence, vocab):
  numerical_sentence = []
  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])
  return numerical_sentence

In [7]:
input_sentences = document.split('\n')
numerical_sequences = []

for sentence in input_sentences:
  tokens = word_tokenize(sentence.lower())
  numerical_sequences.append(text_to_indices(tokens, vocab))

print("Sentence count:", len(input_sentences))
print("Numerical sequence count:", len(numerical_sequences))

Sentence count: 78
Numerical sequence count: 78


### Generate Training Sequences

In [8]:
training_sequences = []
for sequence in numerical_sequences:
  for i in range(1, len(sequence)):
    training_sequences.append(sequence[:i+1])
    
print("Training sequence count:", len(training_sequences))
training_sequences[:20]

Training sequence count: 942


[[1, 2],
 [1, 2, 3],
 [4, 5],
 [4, 5, 2],
 [4, 5, 2, 6],
 [4, 5, 2, 6, 7],
 [4, 5, 2, 6, 7, 8],
 [4, 5, 2, 6, 7, 8, 9],
 [4, 5, 2, 6, 7, 8, 9, 10],
 [4, 5, 2, 6, 7, 8, 9, 10, 11],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14, 15],
 [2, 6],
 [2, 6, 16],
 [2, 6, 16, 17],
 [2, 6, 16, 17, 18],
 [2, 6, 16, 17, 18, 19]]

### Padding Training Sequences

In [9]:
seq_lengths = []
for sequence in training_sequences:
  seq_lengths.append(len(sequence))

max_seq_length = max(seq_lengths)
print("Max sequence length:", max_seq_length)

padded_training_sequence = []
for sequence in training_sequences:
  padding_length = max_seq_length - len(sequence)
  padded_training_sequence.append([0]*padding_length + sequence)
  
print("Padded Training Sequence length:", len(padded_training_sequence[10]))

Max sequence length: 62
Padded Training Sequence length: 62


### Split the Features and Labels

In [10]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)
print("Padded Training Sequence shape:", padded_training_sequence.shape)

X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:,-1]
print("X shape:", X.shape)
print("y shape:", y.shape)

Padded Training Sequence shape: torch.Size([942, 62])
X shape: torch.Size([942, 61])
y shape: torch.Size([942])


In [11]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   0,   0,   4],
        ...,
        [  0,   0,   0,  ...,   0, 285, 176],
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287]])

In [12]:
y[:10]

tensor([ 2,  3,  5,  2,  6,  7,  8,  9, 10, 11])

### Defining the DataLoader

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
class MyDataset(Dataset):

  def __init__(self, X, y):
    self.X = X.to(device)
    self.y = y.to(device)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [15]:
train_dataset = MyDataset(X, y)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

### Test the DataLoder

In [16]:
for batch_idx, (batch_x, batch_y) in enumerate(train_dataloader):
  print(f"batch: {batch_idx}, {batch_x.shape}, {batch_y.shape}")
  print("\t", batch_x, batch_y)
  if batch_idx >= 1:
    break

batch: 0, torch.Size([32, 61]), torch.Size([32])
	 tensor([[  0,   0,   0,  ...,   0,  22,  65],
        [  0,   0,   0,  ...,  78, 187, 135],
        [  0,   0,   0,  ...,  94, 241,  45],
        ...,
        [  0,   0,   0,  ...,   1,   2, 125],
        [  0,   0,   0,  ...,  19, 176, 223],
        [  0,   0,   0,  ..., 175,  30,  68]], device='cuda:0') tensor([ 66,  86, 163,  27,  36,   8, 258,   2,   2,  23,   2,  87,  89,  93,
        202,  16, 262,   7, 154,   2, 212, 176,  25, 151,  57,  78, 155,  17,
          5, 102, 186,   5], device='cuda:0')
batch: 1, torch.Size([32, 61]), torch.Size([32])
	 tensor([[  0,   0,   0,  ...,  17, 242, 176],
        [  0,   0,   0,  ...,  81, 267, 252],
        [  0,   0,   0,  ...,  22,  23, 131],
        ...,
        [  0,   0,   0,  ...,  27,   2,   6],
        [  0,   0,   0,  ...,   0,  22,  23],
        [  0,   0,   0,  ..., 201,   2, 149]], device='cuda:0') tensor([242, 268, 164, 127,  38, 190,   5, 263, 146,  22,  23,  35, 246,  97,
    

### Design the Model

In [17]:
class SimpleLSTM(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=100)
    #self.rnn = nn.RNN(100, 150, batch_first=True)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    #self.gru = nn.GRU(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    #internal_states, hidden_state = self.rnn(embedded)
    internal_states, (hidden_state, cell_state) = self.lstm(embedded)
    #internal_states, hidden_state = self.gru(embedded)
    logits = self.fc(hidden_state.squeeze(0))
    return logits

In [18]:
epochs = 50
learning_rate = 0.001

model = SimpleLSTM(len(vocab)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model_stats = summary(model, input_data=batch_x, verbose=2, col_width=16,
                      col_names=["kernel_size", "input_size", "output_size", "num_params"])

Layer (type:depth-idx)                   Kernel Shape     Input Shape      Output Shape     Param #
SimpleLSTM                               --               [32, 61]         [32, 289]        --
├─Embedding: 1-1                         --               [32, 61]         [32, 61, 100]    28,900
│    └─weight                            [100, 289]                                         └─28,900
├─LSTM: 1-2                              --               [32, 61, 100]    [32, 61, 150]    151,200
│    └─weight_ih_l0                      [600, 100]                                         ├─60,000
│    └─weight_hh_l0                      [600, 150]                                         ├─90,000
│    └─bias_ih_l0                        [600]                                              ├─600
│    └─bias_hh_l0                        [600]                                              └─600
├─Linear: 1-3                            --               [32, 150]        [32, 289]        43,639
│    └─w

### Analyze The Model Layers

In [19]:
emb_layer_output = model.embedding(batch_x)
print("Input sequence shape\t  :", batch_x.shape)
print("Embedding output shape\t  :", emb_layer_output.shape)

if hasattr(model, "rnn"):   
    internal_states, hidden_state = model.rnn(emb_layer_output)
    print("\nRNN internal states shape:", internal_states.shape)
    print("RNN output state shape\t :", hidden_state.shape)
if hasattr(model, "lstm"):
    internal_states, (hidden_state, cell_state)= model.lstm(emb_layer_output)
    print("\nLSTM internal states shape:", internal_states.shape)
    print("LSTM hidden state shape\t  :", hidden_state.shape)
    print("LSTM cell state shape\t  :", cell_state.shape)
if hasattr(model, "gru"):   
    internal_states, hidden_state = model.gru(emb_layer_output)
    print("\nGRU internal states shape:", internal_states.shape)
    print("GRU output state shape\t :", hidden_state.shape)

fc_layer_output = model.fc(hidden_state.squeeze(0))
print("\nFully Connected output shape:", fc_layer_output.shape)

Input sequence shape	  : torch.Size([32, 61])
Embedding output shape	  : torch.Size([32, 61, 100])

LSTM internal states shape: torch.Size([32, 61, 150])
LSTM hidden state shape	  : torch.Size([1, 32, 150])
LSTM cell state shape	  : torch.Size([1, 32, 150])

Fully Connected output shape: torch.Size([32, 289])


### Train the Model

In [20]:
for epoch in range(epochs):
  epoch_loss = 0
  model = model.train()  
  for batch_idx, (batch_x, batch_y) in enumerate(train_dataloader):
    # forward pass
    logits = model(batch_x)
    loss = criterion(logits, batch_y)
    
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    # if not batch_idx % 8:
    #   print(f' -> batch {batch_idx+1:03d} | loss: {loss:.2f}')
  
  print(f'Epoch: {epoch+1:03d}/{epochs:03d} | epoch_loss: {epoch_loss:.2f}')

Epoch: 001/050 | epoch_loss: 166.90
Epoch: 002/050 | epoch_loss: 145.94
Epoch: 003/050 | epoch_loss: 132.18
Epoch: 004/050 | epoch_loss: 119.31
Epoch: 005/050 | epoch_loss: 106.40
Epoch: 006/050 | epoch_loss: 94.58
Epoch: 007/050 | epoch_loss: 83.77
Epoch: 008/050 | epoch_loss: 74.32
Epoch: 009/050 | epoch_loss: 65.67
Epoch: 010/050 | epoch_loss: 57.34
Epoch: 011/050 | epoch_loss: 49.97
Epoch: 012/050 | epoch_loss: 43.83
Epoch: 013/050 | epoch_loss: 38.60
Epoch: 014/050 | epoch_loss: 33.15
Epoch: 015/050 | epoch_loss: 29.26
Epoch: 016/050 | epoch_loss: 25.77
Epoch: 017/050 | epoch_loss: 22.66
Epoch: 018/050 | epoch_loss: 20.11
Epoch: 019/050 | epoch_loss: 17.73
Epoch: 020/050 | epoch_loss: 15.91
Epoch: 021/050 | epoch_loss: 14.47
Epoch: 022/050 | epoch_loss: 13.12
Epoch: 023/050 | epoch_loss: 11.95
Epoch: 024/050 | epoch_loss: 11.29
Epoch: 025/050 | epoch_loss: 10.11
Epoch: 026/050 | epoch_loss: 9.32
Epoch: 027/050 | epoch_loss: 8.79
Epoch: 028/050 | epoch_loss: 8.34
Epoch: 029/050 | e

### Make Prediction

In [21]:
def predict(model, vocab, input_text):
  tokenized_text = word_tokenize(input_text.lower())
  numerical_text = text_to_indices(tokenized_text, vocab)
  padding_length = max_seq_length - 1 - len(numerical_text)
  padded_text = torch.tensor([0]*padding_length + numerical_text, dtype=torch.long).unsqueeze(0)
  logits = model(padded_text.to(device))
  logit, index = torch.max(logits, dim=1)
  return list(vocab.keys())[index], logit.item()

In [22]:
prediction, confidence = predict(model, vocab, "The course follows a monthly")
print(f"Confidence: {confidence:.2f}")
print(f"Answer: {prediction}")

Confidence: 10.83
Answer: subscription


In [23]:
num_tokens = 12
input_prompt = "The course follows a monthly"

for _ in range(num_tokens):
  prediction, logit = predict(model, vocab, input_prompt)  
  input_prompt += " " + prediction
  print(input_prompt)

The course follows a monthly subscription
The course follows a monthly subscription model
The course follows a monthly subscription model where
The course follows a monthly subscription model where you
The course follows a monthly subscription model where you have
The course follows a monthly subscription model where you have to
The course follows a monthly subscription model where you have to make
The course follows a monthly subscription model where you have to make monthly
The course follows a monthly subscription model where you have to make monthly payments
The course follows a monthly subscription model where you have to make monthly payments of
The course follows a monthly subscription model where you have to make monthly payments of rs
The course follows a monthly subscription model where you have to make monthly payments of rs 799/month


### Evaluate the Model

In [24]:
def calculate_accuracy(model, dataloader):
    correct = 0
    total = 0
    model.eval()
    with torch.inference_mode():
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            probs = model(batch_x)
            _, predicted = torch.max(probs, dim=1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total
    return accuracy

test_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
accuracy = calculate_accuracy(model, test_dataloader)
print(f"Train Accuracy: {accuracy:.2f}")

Train Accuracy: 0.96
